In [1]:
import pandas as pd
data=pd.read_csv("Wholesale customers data.csv",sep=",")

data

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204
436,1,3,39228,1431,764,4510,93,2346
437,2,3,14531,15488,30243,437,14841,1867
438,1,3,10290,1981,2232,1038,168,2125


In [2]:
X=pd.DataFrame(data.drop(["Channel","Region"], axis=1))
X=X.astype(float)
y=data["Region"]

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fresh             440 non-null    float64
 1   Milk              440 non-null    float64
 2   Grocery           440 non-null    float64
 3   Frozen            440 non-null    float64
 4   Detergents_Paper  440 non-null    float64
 5   Delicassen        440 non-null    float64
dtypes: float64(6)
memory usage: 20.8 KB


In [3]:
classes=set(y)

classes

{1, 2, 3}

взять не сильнндатасет с неочень большим кол-вом классов, 3 класса оптимально



In [4]:
df = X
df['y_true'] = y
bts = []
for i in range(3):
  df_bt = df.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])

In [5]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

models = [RandomForestClassifier(), KNeighborsClassifier(), DecisionTreeClassifier()]
preds = []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index=X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))

1.0
0.7363636363636363
1.0


In [6]:
preds_df=pd.concat(preds,axis=1)
preds_df.head()

,0,0,0
0,NaN,3.0,3.0
1,3.0,3.0,NaN
2,NaN,3.0,3.0
3,3.0,3.0,NaN
4,NaN,NaN,3.0


In [7]:
preds_df.to_csv('preds.csv')

In [8]:
!pip install owlready2
from owlready2 import *
onto=get_ontology('onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 43.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for owlready2: filename=Owlready2-0.39-cp38-cp38-linux_x86_64.whl size=22260886 sha256=8397943cecc3a49ec4c661526de488f46b4cf0a01160af0cb39afa869b3e2a03
  Stored in directory: /root/.cache/pip/wheels/bd/16/5f/a0bfc34a8f7682cbd6b4d9cb1436c0a0a04ac3579394d7e28a
Successfully built owlready2


In [9]:
import types

with onto:
  tsr= types.new_class('tsr',(ObjectProperty,TransitiveProperty,SymmetricProperty,ReflexiveProperty))
  ts=types.new_class('ts',(ObjectProperty,TransitiveProperty,SymmetricProperty))

  for cl in classes:
    onto_cl=types.new_class(f'Class{cl}',(Thing,))
    onto_cl_know=types.new_class(f'Know_Class{cl}',(Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')


In [10]:
with onto:
  for index, row in preds_df.iterrows():
    inst_old = None
    for j, pred in enumerate(preds_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto.owl', format='ntriples')

In [11]:
with onto:
  sync_reasoner_pellet(infer_property_values= True,debug=2)

onto.save('reasoned.owl',format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python

* Owlready * Adding relation onto.inst423_1 tsr onto.inst423_1
* Owlready * Adding relation onto.inst423_1 ts onto.inst423_1
* Owlready * Adding relation onto.inst435_2 tsr onto.inst435_2
* Owlready * Adding relation onto.inst314_1 tsr onto.inst314_1
* Owlready * Adding relation onto.inst314_1 ts onto.inst314_1
* Owlready * Adding relation onto.inst327_2 tsr onto.inst327_2
* Owlready * Adding relation onto.inst134_1 tsr onto.inst134_1
* Owlready * Adding relation onto.inst134_1 ts onto.inst134_1
* Owlready * Adding relation onto.inst432_1 tsr onto.inst432_1
* Owlready * Adding relation onto.inst432_1 ts onto.inst432_1
* Owlready * Adding relation onto.inst23_2 tsr onto.inst23_2
* Owlready * Adding relation onto.inst23_2 tsr onto.inst23_0
* Owlready * Adding relation onto.inst23_2 ts onto.inst23_2
* Owlready * Adding relation onto.inst23_2 ts onto.inst23_0
* Owlready * Adding relation onto.inst301_2 tsr onto.inst301_2
* Owlready * Adding relation onto.inst301_2 ts onto.inst301_2
* Owlre

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [12]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

onto.save('reasoned_cwa.owl',format='ntriples')

In [13]:
with onto:
  sync_reasoner_pellet(debug=2)

onto.save('reasoned_cwa.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python